# Prepare Workspace

In [1]:
import pandas as pd

In [2]:
# Load datasets explicitly
airports = pd.read_csv('/Users/michael/Desktop/CS506 Project/Airports.csv', skiprows=1)
flights = pd.read_csv('/Users/michael/Desktop/CS506 Project/Flights.csv')
weather = pd.read_csv('/Users/michael/Desktop/CS506 Project/Weather.csv')

# Step 1: Clean and Prepare Airports and Weather Datasets

In [3]:
# Explicitly clean and prepare Airports DataFrame to map ICAO to IATA clearly
airport_codes = airports[['icao', 'code']].copy()
airport_codes.columns = ['ICAO', 'IATA']
airport_codes['ICAO'] = airport_codes['ICAO'].str.strip().str.upper()
airport_codes['IATA'] = airport_codes['IATA'].str.strip().str.upper()

# Clean Weather dataset explicitly and prepare ICAO and date columns
weather['AirportCode'] = weather['AirportCode'].str.strip().str.upper()
weather['Date'] = pd.to_datetime(weather['StartTime(UTC)']).dt.date

# Step 2: Merge Weather and Airports for IATA Codes

In [4]:
# Merge weather explicitly with airports to get IATA codes
weather_with_iata = weather.merge(
    airport_codes,
    left_on='AirportCode',
    right_on='ICAO',
    how='left'
)

# Step 3: Clean Flights Dataset

In [5]:
# Clean flights dataset explicitly
flights['ORIGIN'] = flights['ORIGIN'].str.strip().str.upper()
flights['DEST'] = flights['DEST'].str.strip().str.upper()
flights['Date'] = pd.to_datetime(flights['FL_DATE']).dt.date

# Step 4: Merge Weather and Flight Datasets

In [6]:
# Merge weather data explicitly with flights data based on ORIGIN airport and Date
merged_origin_weather = flights.merge(
    weather_with_iata,
    left_on=['ORIGIN', 'Date'],
    right_on=['IATA', 'Date'],
    how='left',
    suffixes=('', '_origin_weather')
).rename(columns={
    'Type': 'WeatherType_Origin',
    'Severity': 'Severity_Origin',
    'Precipitation(in)': 'Precipitation_Origin'
})

# Merge weather data explicitly again with flights data for DEST airport
merged_full_weather = merged_origin_weather.merge(
    weather_with_iata,
    left_on=['DEST', 'Date'],
    right_on=['IATA', 'Date'],
    how='left',
    suffixes=('', '_dest_weather')
).rename(columns={
    'Type': 'WeatherType_Dest',
    'Severity': 'Severity_Dest',
    'Precipitation(in)': 'Precipitation_Dest'
})

# Step 5: Clean and Finalise

In [7]:
# Drop redundant columns explicitly to clearly finalize dataset
final_columns_to_drop = ['AirportCode', 'ICAO', 'IATA', 'AirportCode_dest_weather', 'ICAO_dest_weather', 'IATA_dest_weather']
final_dataset = merged_full_weather.drop(columns=final_columns_to_drop, errors='ignore')

# Save explicitly the final cleaned merged dataset
final_dataset.to_csv('final_flights_weather_merged.csv', index=False)

# Also explicitly output a smaller file for quick verification (first 50 rows)
final_dataset.head(50).to_csv('small_sample_flights_weather.csv', index=False)

# Explicitly verify the merged dataset clearly
print("✅ Successfully merged Flights and Weather datasets. Here's a preview:")
print(final_dataset.head())
print(f"\nFinal merged dataset dimensions: {final_dataset.shape}")
print("✅ Small verification file 'small_sample_flights_weather.csv' created (first 50 rows).")

✅ Successfully merged Flights and Weather datasets. Here's a preview:
      FL_DATE                AIRLINE                AIRLINE_DOT AIRLINE_CODE  \
0  2019-01-09  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
1  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
2  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
3  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
4  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   

   DOT_CODE  FL_NUMBER ORIGIN          ORIGIN_CITY DEST    DEST_CITY  ...  \
0     19977       1562    FLL  Fort Lauderdale, FL  EWR   Newark, NJ  ...   
1     19790       1149    MSP      Minneapolis, MN  SEA  Seattle, WA  ...   
2     19790       1149    MSP      Minneapolis, MN  SEA  Seattle, WA  ...   
3     19790       1149    MSP      Minneapolis, MN  SEA  Seattle, WA  ...   
4     19790       1149    MSP      Minneapolis, MN  SEA  Seattle